In [2]:
import datetime
import pandas as pd
import requests
import io
import json

url = 'http://gis.sutran.gob.pe/alerta_sutran/script_cgm/carga_xlsx.php'
headers = {'Accept': '/',
'Accept-Encoding': 'gzip, deflate',
'Accept-Language': 'en,es;q=0.9,en-GB;q=0.8',
'Connection': 'keep-alive',
'Content-Length': '9',
'Content-type': 'application/x-www-form-urlencoded; charset=UTF-8-SIG',
'Host': 'gis.sutran.gob.pe',
'Origin': 'http://gis.sutran.gob.pe',
'Referer': 'http://gis.sutran.gob.pe/alerta_sutran/',
'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Mobile Safari/537.36'}
body = {'tipo':'MAPA'}
req = requests.post(url, headers=headers, data=body)
now = datetime.datetime.now()

my_json = json.load(io.BytesIO(req.content))

restringido = {}
restringido['data'] = my_json['restringido']

interrumpido = {}
interrumpido['data'] = my_json['interrumpido']

with open('restringido.json', 'w') as f:
    json.dump(restringido, f)

with open('interrumpido.json', 'w') as f:
    json.dump(interrumpido, f)

df_r = pd.read_json('restringido.json', encoding = 'utf-8-sig', orient='split')
df_restringido = pd.json_normalize(df_r['properties'])

df_i = pd.read_json('interrumpido.json', encoding = 'utf-8-sig', orient='split')
df_interrumpido = pd.json_normalize(df_i['properties'])

frames = [df_restringido, df_interrumpido]
df_total = pd.concat(frames, ignore_index=True)
df_total = df_total[df_total['motivo'] == 'HUMANO']
df_total = df_total.replace('TRANSITO', 'TRÁNSITO', regex=True)
df_total = df_total.replace('"', '', regex=True)
df_total['style'] = df_total['estado'].str.split(' ').str[1].str.lower()
df_total.to_csv(f'{now}.csv', index=False)
df_total

,item,visualizacion_,tipo_alerta_,estado,fecha_evento,fecha_actualizacion,ubigeo,afectacion,evento,latitud,...,_sort_,_fecha_actualizacion_,cant_vehiculos_detenidos_pasajeros,cant_vehiculos_detenidos_mercancias,fuente,motivo,codigo_via,nombre_carretera,alerta_activa_24_horas,style
2,283,SI,VIGENTE,TRÁNSITO RESTRINGIDO,08/01/2023,10/02/2023 15:30 HORAS,MADRE DE DIOS/TAMBOPATA/TAMBOPATA,KM 402,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-12.6898325,...,-12.6898325+-69.4042951,2023 15:30 HORAS+02+10+,0,0,IIRSA SUR,HUMANO,PE-30C,CARRETERA URCOS - MAZUCO - IÑAPARI,SI,restringido
7,29,SI,VIGENTE,TRÁNSITO RESTRINGIDO,07/02/2023,10/02/2023 15:30 HORAS,APURIMAC/ABANCAY/PICHIRHUA,KM 765,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-13.68223922,...,-13.68223922+-72.95762268,2023 15:30 HORAS+02+10+,0,0,UNIPRCAR ABANCAY,HUMANO,PE-3S,LONGITUDINAL DE LA SIERRA SUR,SI,restringido
8,56,SI,VIGENTE,TRÁNSITO RESTRINGIDO,05/01/2023,10/02/2023 15:30 HORAS,APURIMAC/AYMARAES/TORAYA,KM 377,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-14.07238288,...,-14.07238288+-73.25490188,2023 15:30 HORAS+02+10+,0,0,CEOPOL,HUMANO,PE-30A,CARRETERA NASCA - PUQUIO - ABANCAY,SI,restringido
9,50,SI,VIGENTE,TRÁNSITO RESTRINGIDO,19/12/2022,10/02/2023 15:30 HORAS,APURIMAC/AYMARAES/COTARUSE,KM 299 + 500,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-14.514706,...,-14.514706+-73.273429,2023 15:30 HORAS+02+10+,0,0,COMISARIA PNP COTARUSE,HUMANO,PE-30A,CARRETERA NASCA - PUQUIO - ABANCAY,SI,restringido
12,282,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,06/01/2023,10/02/2023 15:30 HORAS,MADRE DE DIOS/TAMBOPATA/LAS PIEDRAS,KM 512+800,(PUENTE MAVILA) PARO INDEFINIDO SOLICITANDO NU...,-11.931417,...,-11.931417+-69.117462,2023 15:30 HORAS+02+10+,0,0,IIRSA SUR,HUMANO,PE-30C,CARRETERA URCOS - MAZUCO - IÑAPARI,SI,interrumpido
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,319,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,05/01/2023,10/02/2023 15:30 HORAS,PUNO/EL COLLAO/SANTA ROSA,KM 175 AL 213,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-16.7991935,...,-16.7991935+-70.0340013,2023 15:30 HORAS+02+10+,0,0,DESPRCAR PNP MAZOCRUZ/CEOPOL,HUMANO,PE-36A,CARRETERA MOQUEGUA - TORATA - MAZOCRUZ - DESAG...,SI,interrumpido
81,314,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,07/01/2023,10/02/2023 15:30 HORAS,PUNO/EL COLLAO/CAPAZO,KM 171 AL KM 226,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-17.19684952581,...,-17.19684952581+-69.759059671272,2023 15:30 HORAS+02+10+,0,0,DESPRCAR PNP MAZOCRUZ/CEOPOL,HUMANO,PE-38,CARRETERA MAZOCRUZ - TORATA,SI,interrumpido
82,115,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,09/02/2023,10/02/2023 15:30 HORAS,CAJAMARCA/JAEN/JAEN,KM 181,CRUCE CHAMAYA PARO INDEFINIDO SOLICITANDO NUEV...,-5.8378719,...,-5.8378719+-78.75384,2023 15:30 HORAS+02+10+,5,14,CEOPOL,HUMANO,PE-04C,CARRETERA CHIPLE - CHAMAYA,SI,interrumpido
84,366,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,09/02/2023,10/02/2023 15:30 HORAS,SAN MARTIN/TOCACHE/UCHIZA,KM 608,SECTOR PACOTA PARO INDEFINIDO SOLICITANDO NUEV...,-8.294375,...,-8.294375+-76.351927,2023 15:30 HORAS+02+10+,3,10,UD SAN MARTIN,HUMANO,PE-5N,LONGITUDINAL DE LA SELVA NORTE,SI,interrumpido
